Detalle de la tarea: Realizar un python  - jupyter en el cual: 

* ~**seleccione cuales sus datos a analizar**~
* ~**indique el origen de es datos, link, descripción**~
* ~**describa el contenido de las columnas**~
* ~**debe establecer la lectura de los datos y carga en un dataframe (utilice pandas)**~
* **debe realizar como mínimo 4 gráficos y estos deben tener la siguiente configuración**
    * **texto con pregunta, explicación del gráfico, la importancia del análisis y los datos involucrados**
    * **instrucciones python para la creación de gráficos estadísticos, sugerencia utilizar pandas y matplotlib**
    * **imagen del gráfico**
* **debe por lo menos tener un gráfico de torta, uno de barra y uno de curvas (pueden incluir más de una categoria, ejemplo: sexo masculino, femenino, otro)**
* **si cree necesario generar más gráficos para explicar su análisis, puede hacerlo**

Nota: el ayudante es Matias Martínez, a quien copio en este email.


===============================================================================================

Se analizaran datos meteorologicos del pais recolectados del año 2023. Los datos seran extraidos del sitio Dirección Meteorológica de Chile - Servicios Climáticos de la Dirección General De Aeronáutica Civil. El sitio contempla una amplia cantidad de datos meteorologicos de diferentes ambitos, recolectados de varias estaciones posicionadas a lo largo del pais.

Link al sitio: https://climatologia.meteochile.gob.cl/

Los datos descargados del sitio vienen en planillas excel separadas con diferentes formatos. Por ello, se ha decidido preprocesar los datos descargados para consolidarlos en archivos que contemplen los datos de interes de una sola estacion o en un solo archivo grande, que contemple las columnas relevantes para el analisis. Para efectos de simplicidad, se han elegido 5 estaciones localizadas a lo largo del pais(Zona norte, centro y sur)

A continuacion se describen las columnas del archivo resultante:
- Fecha y hora del registro
- La estación meteorologica de la cual se extrajo el dato, denotada por el identificador de dicha estacion
- Radiacion Global Instantanea, medida en watt/m2
- Temperatura, medida en celsius
- Agua caida en un periodo de 6 horas, medida en mm
- Velocidad del viento, medida en nudos(kt)
- Nivel de humedad relativa del aire, denotada por un porcentaje.

===============================================================================================

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

#Lectura de identificadores
#Como los archivos descargados siguen un formato que involucra el identificador, es util tener a mano estos datos para agrupar los datos

f = open("Datos/ID_Estaciones.txt", "r")
ids = []
lines = f.readlines()

for line in lines:
    ids.append(line.split(",")[2].split("\n")[0])

print(ids)

['510005', '400009', '180005', '330020', '550001']


In [5]:
#Agrupacion de datos.
#
#Se agrupa los diferentes archivos de datos descargados en uno solo, con el proposito de facilitar la lectura y el acceso de los datos para el futuro.

for id in ids:
    
    df = pd.read_csv("Datos/"+ id +"/"+ id +"_2023_Agua6Horas_.csv", delimiter=";")
    df2 = pd.read_csv("Datos/"+ id +"/"+ id +"_2023_Humedad_.csv", delimiter=";")
    df3 = pd.read_csv("Datos/"+ id +"/"+ id +"_2023_Temperatura_.csv", delimiter=";")
    df4 = pd.read_csv("Datos/"+ id +"/"+ id +"_2023_Viento_.csv", delimiter=";")
    
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    
    df5 = pd.DataFrame(columns=['momento', 'RadGInst (watt/m2)'])
    
    for x in meses:
        tempDf = pd.DataFrame(columns=['momento', 'RadGInst (watt/m2)'])
        fragDf = pd.read_html("Datos/"+ id +"/Radiacion/"+x+".xls")[0]
        tempDf['momento'] = fragDf['Fecha'].astype(str) + " " + fragDf['Hora (UTC)'] +":00"
        tempDf['RadGInst (watt/m2)'] = fragDf['RadGInst (watt/m2)']
        if(df5.size == 0):
            df5 = pd.concat([tempDf], ignore_index = True)
        else:
            df5 = pd.concat([df5,tempDf], ignore_index = True)
    
    df5["index"] = id
        
    dfArr = [df,df2,df3,df4,df5]
    
    maxDf = dfArr[0]
    
    for x in dfArr:
        if(x.iloc[:,0].size > maxDf.iloc[:,0].size):
            maxDf = x
            
    if(maxDf.columns.values[0] != "momento"):
        cleanDf = maxDf.reset_index().iloc[:,:2]
        cleanDf.columns.values[1] = "momento"
    else:
        cleanDf = maxDf.iloc[:,[0,2]]
        
    newFrame = ["HR_Valor", "RRR6_Valor", "ff_Valor", "T_Valor", "RadGInst(watt/m2)"]
    
    col = df.reset_index().iloc[:,1:3]
    col1 = df2.reset_index().iloc[:,1:]
    col2 = df3.reset_index().iloc[:,1:]
    col3 = df4.reset_index().iloc[:,[1,3]]
    col4 = df5.iloc[:,:2]
    
    col.rename(columns = {'level_1':'momento'}, inplace = True) 
    col1.rename(columns = {'CodigoNacional':'momento'}, inplace = True) 
    col2.rename(columns = {'CodigoNacional':'momento'}, inplace = True) 
    col3.rename(columns = {'level_1':'momento'}, inplace = True) 
        
    df_merged = cleanDf.merge(col, on = 'momento', how = 'left')
    df_merged = df_merged.merge(col1, on = 'momento', how = 'left')
    df_merged = df_merged.merge(col2, on = 'momento', how = 'left')
    df_merged = df_merged.merge(col3, on = 'momento', how = 'left')
    
    df_merged = df_merged.merge(col4, on = 'momento', how = 'left')

    df_merged.columns.values[0] = "ID_Estacion"
    df_merged.columns.values[2] = "RR6_Valor"
    df_merged.columns.values[3] = "HR_Valor"
    df_merged.columns.values[4] = "Ts_Valor"
    df_merged.columns.values[5] = "ff_Valor"
    
    df_merged.to_excel(id+"_agrupado.xlsx")
    print('DataFrame de id: '+id+' generado con exito')

DataFrame de id: 510005 generado con exito
DataFrame de id: 400009 generado con exito
DataFrame de id: 180005 generado con exito
DataFrame de id: 330020 generado con exito
DataFrame de id: 550001 generado con exito
